# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [15]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}
# get OpenAI API key, define LLM configuration

## Define an AutoGen agent

In [16]:
from autogen import ConversableAgent

# create 1st ConversableAgent, basic setup
agent = ConversableAgent(
    name="chatbot", # call this chatbot
    llm_config=llm_config, 
    human_input_mode="NEVER", #not seek for human import, only use model
)

In [17]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!


In [18]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Sure! Please provide me with the joke you would like me to repeat.


## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [19]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.", 
    # without message would only reply in generic purpose, customize the behavior of agent
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [20]:
# initiate the message
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Sure thing, let's keep the laughs coming. So, I recently started a new job as a stand-up comedian.

I was hoping to hit the ground running, but turns out I kept tripping over the punchlines!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, it's all part of the learning curve, Cathy. Just remember, even if you stumble, laughter is the best medicine!

--------------------------------------------------------------------------------
cathy (to joe):

Exactly, Joe! Laughter is the best medicine, until you realize you're laughing at your own jokes and questioning your sanity. But hey, as long as the audience is laughing too, I'll just embrace my inner weirdness!

--------------------------------------------------------------------------------


## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [21]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': "Hey Joe! Sure thing, let's keep the laughs coming. So, I "
             'recently started a new job as a stand-up comedian.\n'
             '\n'
             'I was hoping to hit the ground running, but turns out I kept '
             'tripping over the punchlines!',
  'role': 'user'},
 {'content': "Haha, it's all part of the learning curve, Cathy. Just remember, "
             'even if you stumble, laughter is the best medicine!',
  'role': 'assistant'},
 {'content': 'Exactly, Joe! Laughter is the best medicine, until you realize '
             "you're laughing at your own jokes and questioning your sanity. "
             "But hey, as long as the audience is laughing too, I'll just "
             'embrace my inner weirdness!',
  'role': 'user'}]


In [22]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'total_cost': 0},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 123,
                                                             'cost': 0.000313,
                                                             'prompt_tokens': 257,
                                                             'total_tokens': 380},
                                      'total_cost': 0.000313}}


In [23]:
pprint.pprint(chat_result.summary)

("Exactly, Joe! Laughter is the best medicine, until you realize you're "
 'laughing at your own jokes and questioning your sanity. But hey, as long as '
 "the audience is laughing too, I'll just embrace my inner weirdness!")


## Get a better summary of the conversation

In [24]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, # control the number of conversation
    summary_method="reflection_with_llm",
    # give summary of conversation
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Sure thing, let's keep the laughs coming. So, I recently started a new job as a stand-up comedian.

I was hoping to hit the ground running, but turns out I kept tripping over the punchlines!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, it's all part of the learning curve, Cathy. Just remember, even if you stumble, laughter is the best medicine!

--------------------------------------------------------------------------------
cathy (to joe):

Exactly, Joe! Laughter is the best medicine, until you realize you're laughing at your own jokes and questioning your sanity. But hey, as long as the audience is laughing too, I'll just embrace my inner weirdness!

--------------------------------------------------------------------------------


In [25]:
pprint.pprint(chat_result.summary)

('Laughter is the best medicine, even if you stumble over punchlines or laugh '
 'at your own jokes. Embrace your inner weirdness and keep the jokes rolling.')


## Chat Termination

Chat can be terminated using a termination conditions.

In [26]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    # termination of conversation
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [27]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, glad to hear you're ready for some laughs! Let's get this comedy train going! What do you call a pile of cats? A meowtain!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! How do you catch a squirrel? Climb a tree and act like a nut!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, nice one, Joe! Here's another one for you: Why did the tomato turn red? Because it saw the salad dressing!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic one, Cathy! Alright, I'll leave you with this joke: Why couldn't the bicycle find its way home? Because it lost its bearings!

----------------------------------------------------

In [28]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was: Why couldn't the bicycle find its way home? Because it lost its bearings!

--------------------------------------------------------------------------------
cathy (to joe):

Thanks! Great joke. Well, I gotta go now. See you next time!

--------------------------------------------------------------------------------
